In [1]:
!lsb_release -a

No LSB modules are available.
Distributor ID:	Ubuntu
Description:	Ubuntu 18.04.5 LTS
Release:	18.04
Codename:	bionic


In [2]:
%%writefile gconf.sh

#!/usr/bin/env sh

sudo apt-get update -y
sudo apt-get upgrade -y
sudo apt-get dist-upgrade -y
sudo apt-get install build-essential software-properties-common -y
sudo add-apt-repository ppa:ubuntu-toolchain-r/test -y
sudo apt-get update -y
sudo apt-get install gcc-9 g++-9 -y
sudo update-alternatives --install /usr/bin/gcc gcc /usr/bin/gcc-9 60 --slave /usr/bin/g++ g++ /usr/bin/g++-9
sudo update-alternatives --config gcc

select gcc-9

Writing gconf.sh


In [3]:
!sudo /bin/bash gconf.sh > /dev/null 2>&1

In [5]:
!# add new tbb library (2018 year required, 2017 is only available)
!echo "deb http://old-releases.ubuntu.com/ubuntu eoan main universe" | sudo tee -a  /etc/apt/sources.list 
!sudo apt update > /dev/null 2>&1

!sudo apt install libstdc++-9-dev > /dev/null 2>&1
!sudo apt install  libtbb-dev > /dev/null 2>&1

deb http://old-releases.ubuntu.com/ubuntu eoan main universe


In [6]:
%%writefile profile.h

#pragma once

#include <chrono>
#include <iostream>
#include <optional>
#include <string>
#include <sstream>

class LogDuration {
public:
  explicit LogDuration(const std::string& msg = "")
    : message(msg + ": ")
    , start(std::chrono::steady_clock::now())
  {
  }

  ~LogDuration() {
    auto finish = std::chrono::steady_clock::now();
    auto dur = finish - start;
    std::ostringstream os;
    os << message
       << std::chrono::duration_cast<std::chrono::milliseconds>(dur).count()
       << " ms" << std::endl;
    std::cerr << os.str();
  }
private:
  std::string message;
  std::chrono::steady_clock::time_point start;
};

#ifndef UNIQ_ID
  #define UNIQ_ID_IMPL(lineno) _a_local_var_##lineno
  #define UNIQ_ID(lineno) UNIQ_ID_IMPL(lineno)
#endif

#define LOG_DURATION(message) \
  LogDuration UNIQ_ID(__LINE__){message};

Writing profile.h


In [7]:
%%writefile sort.cpp
#include "profile.h"
#include <algorithm>
#include <execution>
#include <random>
#include <vector>

using namespace std;

vector<int> GenerateNumbers(mt19937& generator,
                            int number_count, int max_value) {
    vector<int> v(number_count);
    for (int& value : v) {
        value = uniform_int_distribution(0, max_value)(generator);
    }
    return v;
}

int main() {
    mt19937 generator;
    const vector<int> v = GenerateNumbers(generator, 10'000'000, 1'000);

    vector<int> v1 = v;
    {
        LOG_DURATION("simple");
        sort(v1.begin(), v1.end());
    }

    vector<int> v2 = v;
    {
        LOG_DURATION("seq");
        sort(execution::seq, v2.begin(), v2.end());
    }

    vector<int> v3 = v;
    {
        LOG_DURATION("par");
        sort(execution::par, v3.begin(), v3.end());
    }
}

Writing sort.cpp


In [10]:
!g++ sort.cpp --std=c++17  -ltbb -lpthread -w && ./a.out

simple: 4610 ms
seq: 5249 ms
par: 3875 ms


In [11]:
%%writefile futures.cpp

#include <future>
#include <iostream>
#include <vector>
#include <mutex>


std::mutex mtx;
int print_with_mutex_exception_lock_guard(int m) {
    std::lock_guard<std::mutex> lck (mtx);
    if (m == 5) throw "Want hugs";
    std::cout << "got: " << m << std::endl;
    return 2 * m;
}

int main() {
    std::vector<std::future<int>> futures;

    for(int i = 0; i < 10; ++i) {
        futures.push_back(std::async(print_with_mutex_exception_lock_guard, i));
    }

    for (auto& fut : futures) {
        fut.wait();
    }

    return 0;
}

Writing futures.cpp


In [13]:
!g++ futures.cpp --std=c++17  -ltbb -lpthread -w && ./a.out

got: 1
got: 2
got: 3
got: 8
got: 4
got: 0
got: 6
got: 7
got: 9
